# FYP ICT3909 - Yolov12 Training
---

**Name:** Andrea Filiberto Lucas  
**ID No:** 0279704L

---

## Automated Dataset Management with Roboflow and Folder Organization

This script automates the process of managing a dataset using Roboflow. It creates necessary directories, checks for installed dependencies, installs them if missing, and downloads the dataset. It also organizes the dataset into a structured folder hierarchy, ensuring everything is ready for further use.

In [2]:
import os
import subprocess
import shutil
import importlib

# Constants for colored output
COLORS = {
    "green": "\033[92m",  # Green text
    "red": "\033[91m",    # Red text
    "reset": "\033[0m"    # Reset to default color
}

# Define the path to the Versions folder and the target subfolder
versions_path = os.path.abspath(os.path.join("..", "Other", "Versions"))
target_subfolder = os.path.join(versions_path, "NGD-Yolov12")

# Check if the Versions folder exists, if not, create it
if not os.path.exists(versions_path):
    os.makedirs(versions_path)
    print(f"[{COLORS['green']}✔{COLORS['reset']}] Folder created at: {versions_path}")

# Check if the MDD-AFL-Yolov12 subfolder exists
if os.path.exists(target_subfolder):
    print(f"[{COLORS['green']}✔{COLORS['reset']}] The subfolder '{target_subfolder}' already exists. Skipping download!")
else:
    # Check if roboflow is installed
    if importlib.util.find_spec("roboflow") is not None:                                                                                                        # type: ignore
        print(f"[{COLORS['green']}✔{COLORS['reset']}] Roboflow is already installed!")
    else:
        # Install roboflow using pip
        try:
            subprocess.check_call(["pip", "install", "roboflow"])
            print(f"[{COLORS['green']}✔{COLORS['reset']}] Roboflow successfully installed!")
        except subprocess.CalledProcessError as e:
            print(f"[{COLORS['red']}✖{COLORS['reset']}]Failed to install Roboflow. Please check your setup.")
            raise e

    # Import and use Roboflow
    from roboflow import Roboflow                                                                                                                               # type: ignore

    # Prompt the user for their API key
    print("Please enter your Roboflow API key to download the dataset...")
    api_key = input("Please enter your Roboflow API key: ")

    # Initialize Roboflow with the provided API key
    rf = Roboflow(api_key=api_key)

    # Retrieve project and version
    project = rf.workspace("ict3909-fyp").project("news-graphic-dataset")
    version = project.version(1)

    # Download the dataset
    dataset = version.download("yolov12")

    current_folder = os.getcwd()  # Get the current working directory
    original_folder = os.path.join(current_folder, "News-Graphic-Dataset-1")
    renamed_folder = os.path.join(current_folder, "NGD-Yolov12")
    target_folder = os.path.join(versions_path, "NGD-Yolov12")

    # Check if the original folder exists   
    if os.path.exists(original_folder):
        # Rename the folder
        os.rename(original_folder, renamed_folder)

        # Move the renamed folder to ../Versions/
        shutil.move(renamed_folder, target_folder)
        print(f"[{COLORS['green']}✔{COLORS['reset']}] Folder downloaded to: {target_folder}")
    else:
        print(f"[{COLORS['red']}✖{COLORS['reset']}]Folder '{original_folder}' does not exist. No action taken.")


[✔] The subfolder 'd:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12' already exists. Skipping download!


## Automated Library Installer in Python

This script automates checking and installing libraries from a JSON file. It verifies installations, installs missing libraries via `pip`, and provides clear, colored output for success or errors. With built-in error handling and preloaded common libraries, it simplifies dependency management in Python projects.

In [ ]:
import json
import importlib.util

# Path to the JSON file
lib_file_path = os.path.join("..", "Other", "Libraries", "Yolo.json")

# Read the libraries from the JSON file
try:
    with open(lib_file_path, 'r') as file:
        libraries = json.load(file)
except FileNotFoundError:
    print(f"{COLORS['red']}Error: Library file not found at {lib_file_path}{COLORS['reset']}")
    exit(1)
except json.JSONDecodeError:
    print(f"{COLORS['red']}Error: Failed to decode JSON from the library file.{COLORS['reset']}")
    exit(1)

# Function to check and install libraries
def check_and_install_libraries(libraries):
    for lib, import_name in libraries.items():
        # Check if the library is installed by checking its module spec
        if importlib.util.find_spec(import_name) is not None:
            print(f"[{COLORS['green']}✔{COLORS['reset']}] Library '{lib}' is already installed.")
        else:
            # If the library is not found, try to install it
            print(f"[{COLORS['red']}✖{COLORS['reset']}] Library '{lib}' is not installed. Installing...")
            try:
                subprocess.check_call(["pip", "install", lib])
                print(f"[{COLORS['green']}✔{COLORS['reset']}] Successfully installed '{lib}'.")
            except subprocess.CalledProcessError:
                print(f"[{COLORS['red']}✖{COLORS['reset']}] Failed to install '{lib}'. Please install it manually.")


# Execute the function to check and install libraries
check_and_install_libraries(libraries)

# Import necessary libraries 
import time
import torch                                                                                                                                                    #type: ignore
import matplotlib.patches as patches                                                                                                                            #type: ignore
from ultralytics import YOLO                                                                                                                                    #type: ignore

[✔] Library 'opencv-python' is already installed.
[✔] Library 'matplotlib' is already installed.
[✔] Library 'tqdm' is already installed.
[✔] Library 'ultralytics' is already installed.


## Determine the Available Device for Computation

This function identifies and selects the best available device for running deep learning models, prioritizing hardware acceleration via CUDA (GPU) or MPS (Metal Performance Shaders). If no hardware acceleration is available, it defaults to the CPU and notifies the user.

In [4]:
# Determine the available device: CUDA, MPS, or CPU.
def determine_device():
    if torch.cuda.is_available():
        device = "cuda"
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    elif torch.backends.mps.is_available():
        device = "mps"
        print("Using MPS (Metal Performance Shaders) for acceleration.")
    else:
        device = "cpu"
        print("Using CPU. Consider enabling GPU or MPS for faster training.")
    return device

## Training process for Yolov12 object detector.
### Experiment Configuration for YOLOv12

Defines the path to the `data.yaml` file and key hyperparameters for conducting experiments with YOLOv12. These settings include the number of epochs, image size, batch size, and a customizable experiment name for version tracking and organization.

In [9]:
# Path to data.yaml
data_yaml_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), "Other", "Versions", "NGD-Yolov12", "data.yaml")

# Prompt user for hyperparameters
try:
    epochs = int(input("Enter the number of epochs: "))
    experiment_number = int(input("Enter the experiment number: "))
    experiment_name = f"NGD-Yolov12_v{experiment_number}"
except ValueError:
    print("Invalid input. Please enter valid numbers for epochs and experiment number.")
    raise

# Display selected parameters
print(f"Selected epochs: {epochs}")
print(f"Experiment name: {experiment_name}")

# Hyperparameters
imgsz = 640
batch_size = 8

Selected epochs: 150
Experiment name: NGD-Yolov12_v4


In [ ]:
# Timer start
start_time = time.time()

# Create or load a YOLOv12 model (from scratch)
try:
    model = YOLO("yolo12.yaml")  # specify architecture
except FileNotFoundError:
    print("Error: YOLOv12 configuration file 'yolo12.yaml' not found. Check setup!")
    raise

# Train
try:
    model.train(
        data=data_yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch_size, # was set to 8
        name=experiment_name,
        pretrained=False,  
        plots=True,
        device = "cuda" if torch.cuda.is_available() else "cpu",
        patience=10,
        dropout=0.4,
        cos_lr=True,
        lr0=0.0001,
        lrf=0.00001,
    )
except Exception as e:
    print(f"Error during training: {e}")
    raise

# Timer end
end_time = time.time()
# Calculate elapsed time
elapsed_time = end_time - start_time
# Convert seconds to hours, minutes, and seconds
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)
seconds = int(elapsed_time % 60)
print(f"Training completed in {hours}h {minutes}m {seconds}s.")

# Evaluate
try:
    metrics = model.val()
    print("Validation Metrics:", metrics)
except Exception as e:
    print(f"Error during validation: {e}")
    raise

WARNING  no model scale passed. Assuming scale='n'.
New https://pypi.org/project/ultralytics/8.3.103 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.101  Python-3.12.7 torch-2.8.0.dev20250405+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: task=detect, mode=train, model=yolo12.yaml, data=d:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\data.yaml, epochs=150, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=NGD-Yolov12_v2, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.4, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=

train: Scanning D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\train\labels... 5250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5250/5250 [00:10<00:00, 483.95it/s]


train: New cache created: D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\train\labels.cache


val: Scanning D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\valid\labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 673.07it/s]

val: New cache created: D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\valid\labels.cache


...

EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 92, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

102 epochs completed in 1.861 hours.
Optimizer stripped from runs\detect\NGD-Yolov12_v2\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\NGD-Yolov12_v2\weights\best.pt, 5.5MB

Validating runs\detect\NGD-Yolov12_v2\weights\best.pt...
Ultralytics 8.3.101  Python-3.12.7 torch-2.8.0.dev20250405+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLO12 summary (fused): 159 layers, 2,557,898 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:02<00:00,  8.25it/s]


                   all        300        943      0.933      0.902      0.941       0.75
 Breaking News Graphic         47         48      0.859      0.917      0.923      0.696
Digital On-Screen Graphic        289        392      0.963      0.917      0.971      0.822
   Lower Third Graphic        147        156      0.949      0.904      0.972      0.785
         News Headline         99        100       0.98      0.986      0.985      0.961
           News Ticker        125        126      0.951      0.929      0.968      0.607
    Other News Graphic         92        121      0.893      0.758      0.829      0.627
Speed: 0.5ms preprocess, 2.2ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\NGD-Yolov12_v2
Training completed in 1h 52m 55s.
Ultralytics 8.3.101  Python-3.12.7 torch-2.8.0.dev20250405+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLO12 summary (fused): 159 layers, 2,557,898 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\valid\labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:02<00:00, 18.25it/s]


                   all        300        943      0.933      0.902      0.941       0.75
 Breaking News Graphic         47         48      0.859      0.917      0.923      0.692
Digital On-Screen Graphic        289        392      0.963      0.917      0.971      0.827
   Lower Third Graphic        147        156      0.955      0.904      0.972      0.785
         News Headline         99        100       0.98      0.986      0.985      0.961
           News Ticker        125        126      0.951      0.929      0.968      0.608
    Other News Graphic         92        121      0.893      0.758      0.829      0.627
Speed: 0.3ms preprocess, 2.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\NGD-Yolov12_v22
Validation Metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000

In [ ]:
# Timer start
start_time = time.time()

# Create or load a YOLOv12 model (from scratch)
try:
    model = YOLO("yolo12.yaml")  # specify architecture
except FileNotFoundError:
    print("Error: YOLOv12 configuration file 'yolo12.yaml' not found. Check setup!")
    raise

# Train
try:
    model.train(
        data=data_yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch_size, # was set to 16
        name=experiment_name,
        pretrained=False,  
        plots=True,
        device = "cuda" if torch.cuda.is_available() else "cpu",
        patience=10,
        dropout=0.3,
        cos_lr=True,
        lr0=0.0001,
        lrf=0.00001,
    )
except Exception as e:
    print(f"Error during training: {e}")
    raise

# Timer end
end_time = time.time()
# Calculate elapsed time
elapsed_time = end_time - start_time
# Convert seconds to hours, minutes, and seconds
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)
seconds = int(elapsed_time % 60)
print(f"Training completed in {hours}h {minutes}m {seconds}s.")

# Evaluate
try:
    metrics = model.val()
    print("Validation Metrics:", metrics)
except Exception as e:
    print(f"Error during validation: {e}")
    raise

WARNING  no model scale passed. Assuming scale='n'.
New https://pypi.org/project/ultralytics/8.3.103 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.101  Python-3.12.7 torch-2.8.0.dev20250405+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: task=detect, mode=train, model=yolo12.yaml, data=d:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\data.yaml, epochs=150, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=NGD-Yolov12_v3, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer

train: Scanning D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\train\labels... 5250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5250/5250 [00:10<00:00, 482.34it/s]


train: New cache created: D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\train\labels.cache


val: Scanning D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\valid\labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 347.62it/s]

val: New cache created: D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\valid\labels.cache


...


83 epochs completed in 0.825 hours.
Optimizer stripped from runs\detect\NGD-Yolov12_v3\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\NGD-Yolov12_v3\weights\best.pt, 5.5MB

Validating runs\detect\NGD-Yolov12_v3\weights\best.pt...
Ultralytics 8.3.101  Python-3.12.7 torch-2.8.0.dev20250405+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLO12 summary (fused): 159 layers, 2,557,898 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:01<00:00,  5.67it/s]


                   all        300        943      0.902      0.891      0.936      0.744
 Breaking News Graphic         47         48      0.863      0.896      0.937      0.702
Digital On-Screen Graphic        289        392      0.958      0.921       0.97      0.809
   Lower Third Graphic        147        156      0.947      0.915      0.963       0.77
         News Headline         99        100      0.951       0.99      0.976      0.943
           News Ticker        125        126      0.874      0.944      0.965      0.606
    Other News Graphic         92        121       0.82      0.678      0.804      0.634
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\NGD-Yolov12_v3
Training completed in 0h 50m 39s.
Ultralytics 8.3.101  Python-3.12.7 torch-2.8.0.dev20250405+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLO12 summary (fused): 159 layers, 2,557,898 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\valid\labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:02<00:00,  9.28it/s]


                   all        300        943      0.903      0.891      0.936      0.745
 Breaking News Graphic         47         48      0.862      0.896      0.938        0.7
Digital On-Screen Graphic        289        392      0.958      0.921       0.97      0.811
   Lower Third Graphic        147        156      0.947      0.915      0.963      0.774
         News Headline         99        100      0.951       0.99      0.976      0.943
           News Ticker        125        126      0.881      0.944      0.965      0.607
    Other News Graphic         92        121      0.821       0.68      0.803      0.634
Speed: 0.4ms preprocess, 2.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\NGD-Yolov12_v32
Validation Metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000

In [10]:
# Timer start
start_time = time.time()

# Create or load a YOLOv12 model (from scratch)
try:
    model = YOLO("yolo12.yaml")  # specify architecture
except FileNotFoundError:
    print("Error: YOLOv12 configuration file 'yolo12.yaml' not found. Check setup!")
    raise

# Train
try:
    model.train(
        data=data_yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch_size, # was set to 8
        name=experiment_name,
        pretrained=False,  
        plots=True,
        device = "cuda" if torch.cuda.is_available() else "cpu",
        patience=10,
        dropout=0.4,
        cos_lr=True,
        lr0=0.0001,
        lrf=0.00001,
    )
except Exception as e:
    print(f"Error during training: {e}")
    raise

# Timer end
end_time = time.time()
# Calculate elapsed time
elapsed_time = end_time - start_time
# Convert seconds to hours, minutes, and seconds
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)
seconds = int(elapsed_time % 60)
print(f"Training completed in {hours}h {minutes}m {seconds}s.")

# Evaluate
try:
    metrics = model.val()
    print("Validation Metrics:", metrics)
except Exception as e:
    print(f"Error during validation: {e}")
    raise

WARNING  no model scale passed. Assuming scale='n'.
New https://pypi.org/project/ultralytics/8.3.103 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.101  Python-3.12.7 torch-2.8.0.dev20250405+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: task=detect, mode=train, model=yolo12.yaml, data=d:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\data.yaml, epochs=150, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=NGD-Yolov12_v4, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.4, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=

train: Scanning D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\train\labels.cache... 5250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5250/5250 [00:00<?, ?it/s]
val: Scanning D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\valid\labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]


Plotting labels to runs\detect\NGD-Yolov12_v4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 113 weight(decay=0.0), 120 weight(decay=0.0005), 119 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\NGD-Yolov12_v4
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/150      2.17G      4.816      5.745       4.51          7        640: 100%|██████████| 657/657 [01:08<00:00,  9.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:01<00:00, 12.55it/s]

                   all        300        943   3.11e-05    0.00275   1.62e-05   1.62e-06


...


102 epochs completed in 1.945 hours.
Optimizer stripped from runs\detect\NGD-Yolov12_v4\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\NGD-Yolov12_v4\weights\best.pt, 5.5MB

Validating runs\detect\NGD-Yolov12_v4\weights\best.pt...
Ultralytics 8.3.101  Python-3.12.7 torch-2.8.0.dev20250405+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLO12 summary (fused): 159 layers, 2,557,898 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:01<00:00, 11.27it/s]


                   all        300        943      0.933      0.902      0.941       0.75
 Breaking News Graphic         47         48      0.859      0.917      0.923      0.696
Digital On-Screen Graphic        289        392      0.963      0.917      0.971      0.822
   Lower Third Graphic        147        156      0.949      0.904      0.972      0.785
         News Headline         99        100       0.98      0.986      0.985      0.961
           News Ticker        125        126      0.951      0.929      0.968      0.607
    Other News Graphic         92        121      0.893      0.758      0.829      0.627
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\NGD-Yolov12_v4
Training completed in 1h 57m 34s.
Ultralytics 8.3.101  Python-3.12.7 torch-2.8.0.dev20250405+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLO12 summary (fused): 159 layers, 2,557,898 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov12\valid\labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:01<00:00, 21.16it/s]


                   all        300        943      0.933      0.902      0.941       0.75
 Breaking News Graphic         47         48      0.859      0.917      0.923      0.692
Digital On-Screen Graphic        289        392      0.963      0.917      0.971      0.827
   Lower Third Graphic        147        156      0.955      0.904      0.972      0.785
         News Headline         99        100       0.98      0.986      0.985      0.961
           News Ticker        125        126      0.951      0.929      0.968      0.608
    Other News Graphic         92        121      0.893      0.758      0.829      0.627
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\NGD-Yolov12_v42
Validation Metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000